In [1]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers
from keras.utils import to_categorical

img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(1024, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(), 
    Dropout(0.5),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

merged_vector = concatenate([encoded_a, encoded_b])

#x = merged_vector
x = Dense(256, activation='relu')(merged_vector)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 2048)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 2048)          0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 256)           2365696     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 512)           0           sequential_1[1][0]      

In [2]:
# data prepare
import cPickle as pickle
import numpy as np
import lmdb
import random

env = lmdb.open("./data/features")
txn = env.begin()

def get_XY(dataset):
    X1 = []
    X2 = []
    Y = []
    for img_a, s_a, img_b, s_b, cmpret in dataset:
        feature_a = txn.get(img_a)
        feature_b = txn.get(img_b)
        if feature_a is None or feature_b is None:
            continue
        feature_a = np.fromstring(feature_a, np.float32)
        feature_b = np.fromstring(feature_b, np.float32)
        X1.append(feature_a)
        X2.append(feature_b)
        Y.append(cmpret)
    X1 = np.array(X1)
    X2 = np.array(X2)
    Y = to_categorical(np.array(Y), num_classes=2)
    return X1, X2, Y

batch_size = 128
train_list = pickle.load(open("./data/train.list", 'rb'))
valid_list = pickle.load(open("./data/valid.list", 'rb'))
X1_train, X2_train, Y_train = get_XY(train_list)
X1_valid, X2_valid, Y_valid = get_XY(valid_list)

env.close()

In [3]:
# training the model
optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([X1_train, X2_train], Y_train, batch_size=batch_size, epochs=10, shuffle=True,
         validation_data=([X1_valid, X2_valid], Y_valid))

Train on 754378 samples, validate on 45499 samples
Epoch 1/10
754378/754378 [==============================] - 332s - loss: 0.6395 - acc: 0.6386 - val_loss: 0.6965 - val_acc: 0.5786
Epoch 2/10
754378/754378 [==============================] - 107s - loss: 0.5579 - acc: 0.7156 - val_loss: 0.7476 - val_acc: 0.5789
Epoch 3/10
754378/754378 [==============================] - 61s - loss: 0.5206 - acc: 0.7445 - val_loss: 0.7614 - val_acc: 0.5617
Epoch 4/10
754378/754378 [==============================] - 57s - loss: 0.4951 - acc: 0.7624 - val_loss: 0.7579 - val_acc: 0.5734
Epoch 5/10
754378/754378 [==============================] - 56s - loss: 0.4797 - acc: 0.7720 - val_loss: 0.7549 - val_acc: 0.5723
Epoch 6/10
754378/754378 [==============================] - 56s - loss: 0.4761 - acc: 0.7741 - val_loss: 0.7459 - val_acc: 0.5732
Epoch 7/10
754378/754378 [==============================] - 56s - loss: 0.4750 - acc: 0.7752 - val_loss: 0.7557 - val_acc: 0.5680
Epoch 8/10
754378/754378 [===========

In [4]:
model.save_weights("./data/rank_model.h5")